In [ ]:
import os  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path  # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import researchpy_fork as rp # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import specification_curve_fork as specy # type:ignore # isort:skip # fmt:skip # noqa # nopep8


# Functions

In [ ]:
# Function to order categories
def categorize_df_results_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        try:
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')
        except ValueError as e:
            print(e)

    return df


# READ DATA

In [ ]:
with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_training.pkl')
assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_manual.shape}')


In [ ]:
with open(f'{data_dir}df_jobs_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

# df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
print(f'Dataframe loaded with shape: {df_jobs.shape}')


In [ ]:
df_manual = categorize_df_results_gender_age(df_manual)

In [ ]:
df_jobs = categorize_df_results_gender_age(df_jobs)

## Set dataframes

In [ ]:
dataframes = {
    'df_manual': df_manual,
    'df_jobs': df_jobs,
}

## Analysis plan:
Below are the possible correct analysis for each df:

#### All DFs:
1. Frequencies
2. Normality tests
   * Normal test
   * Kurtosis test
   * Shapiro
   * Anderson
   * Bartlett
   * Correlation between independent variables (ivs) and control variables - Multicolinarity (pearsonr, spearmanr, kendalltau, and pointbiserialr)
     - **ivs_dummy** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
     - **% Sector per Workforce** (continous ratio) = Sector percentage per worksforce (0-100)
     - **num_words** (continous ratio) = Number of words in job description
     - **English Requirement in Sentence** (binary nominal) = English requirement in job description (0 vs. 1)
     - **Dutch Requirement in Sentence** (binary nominal) = Dutch requirement in job description (0 vs. 1)

#### DF Manual:
1. Chi-square
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
2. One-way ANOVA, interactions, and post-hoc test
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
     - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
3. Logistic Regression  with all interaction (smf):
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
4. Logistic Specification Curve Analysis:
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)

#### DF Jobs:
1. Chi-square
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
2. One-way ANOVA, interactions, and post-hoc test
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
     - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
3. Logistic Regression with all interaction (smf):
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
4. OLS Regression with all interaction:
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
5. Multilevel OLS Regression with all interaction:
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
6. Logistic Specification Curve Analysis:
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
7. OLS Specification Curve Analysis:
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)


# Frequencies


In [ ]:
for df_name, df in dataframes.items():
    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')

    data_names = dvs[:]+ivs_dummy_and_perc[:]

    if df_name == 'df_manual':
        dvs_ = dvs[:]
    elif df_name == 'df_jobs':
        data_names.extend(dvs_prob[:])
        dvs_ = dvs_all[:]

    print('~'*20)
    # print(rp.codebook(df[data_names]))

    # Gender and Age
    print('-'*20)
    print(f'Categorical Summary {ivs}')
    freq_iv=rp.summary_cat(df[ivs]).round(3)
    print(freq_iv)
    freq_iv.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age.csv')
    print('-'*20)
    print('\n')

    # Gender and Age Percentages
    print('-'*20)
    print(f'Continuous Summary {ivs_perc}')
    freq_iv_perc=rp.summarize(df[ivs_perc], ci_level = 0.95, decimals = 3)
    print(freq_iv_perc)
    freq_iv_perc.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age Percentages.csv')
    print('-'*20)
    print('\n')

    # Gender and Age Percentages
    print('-'*20)
    print(f'Continuous Summary {ivs_count}')
    freq_iv_count=rp.summarize(df[ivs_count], ci_level = 0.95, decimals = 3)
    print(freq_iv_count)
    freq_iv_count.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age Counts.csv')
    print('-'*20)
    print('\n')

    # Warmth and Competence
    print('-'*20)
    print(f'Categorical Summary {dvs}')
    freq_dv=rp.summary_cat(df[dvs]).round(3)
    print(freq_dv)
    freq_dv.to_csv(f'{table_save_path}frequencies {df_name} - Warmth and Competence.csv')
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        # Warmth and Competence Probabilities
        print('-'*20)
        print(f'Continuous Summary {dvs_prob}')
        freq_dv_prob=rp.summarize(df[dvs_prob], ci_level = 0.95, decimals = 3)
        print(freq_dv_prob)
        freq_dv_prob.to_csv(f'{table_save_path}frequencies {df_name} - Warmth and Competence Probabilities.csv')
        print('-'*20)
        print('\n')

    print('-'*20)
    print('Grouped Frequencies/ Summary ANOVAs')
    summary_aova = rp.summary_cont(df.groupby(ivs)[dvs], conf=0.95, decimals=3)
    print(summary_aova)
    summary_aova.to_csv(f'{table_save_path}summary anova {df_name} - {ivs} x {dvs}.csv')
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        print('-'*20)
        print('Grouped Frequencies/ Summary ANOVAs')
        summary_aova_probs = rp.summary_cont(df.groupby(ivs)[dvs_all], conf=0.95, decimals=3)
        print(summary_aova_probs)
        summary_aova_probs.to_csv(f'{table_save_path}summary anova probabilities {df_name} - {ivs} x {dvs_all}.csv')
        print('-'*20)
        print('\n')

    # Histogram
    df[ivs_perc].hist()
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    df[ivs_count].hist()
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    df[ivs_dummy].hist()
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        # Histogram
        df[dvs_prob].hist()
        plt.ion()
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
        print('-'*20)
        print('\n')

    # QQ plot
    qq_plot = pg.qqplot(df[ivs_perc], dist='norm')
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    qq_plot = pg.qqplot(df[ivs_count], dist='norm')
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    qq_plot = pg.qqplot(df[ivs_dummy], dist='norm')
    plt.ion()
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        # QQ plot dvs_prob
        qq_plot = pg.qqplot(df[dvs_prob], dist='norm')
        plt.ion()
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
        print('-'*20)
        print('\n')


# Normality Tests


In [ ]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_manual':
        dvs_ = dvs
    elif df_name == 'df_jobs':
        dvs_ = dvs_all

    # Test of Normality for skew and kurtosis
    # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
    norm = scipy.stats.normaltest(df[dvs_])
    print('='*80)
    print(f'{dvs_} Test of Normality:')
    print('-'*80)
    for key, val in dict(zip(normality_tests_labels, norm)).items():
        print(key,': ', val) # not significant
    print('\n')

    # Shapir-Wilk Test of Normality
    # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
    norm_res = scipy.stats.shapiro(df[dvs_])
    print('='*80)
    print(f'{dvs_} Shapir-Wilk Test of Normality:')
    print('-'*80)
    for key, val in dict(zip(normality_tests_labels, norm_res)).items():
        print(key,': ', val) # significant
    print('\n')

    for iv, dv in tqdm_product(ivs_dummy_and_perc, dvs_):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # Anderson-Darling Test of Normality
        # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
        norm_and = scipy.stats.anderson(df[dv])
        print('='*80)
        print('Anderson-Darling Test of Normality:')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # not significant
        print('\n')
        if norm_and.fit_result.success:
            print('Anderson-Darling Test of Normality: The test was successful.')
        else:
            print('Anderson-Darling Test of Normality: The test was not successful.')
        print('~' * 20)
        print('\n')

        # NORMALITY TESTS
        print('NORMALITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        norm = pg.normality(data=df, dv=dv, group=iv).round(3)
        normal = bool(norm['normal'].to_string(index=False))
        print(f"{iv} x {dv} Normality test:\n{norm}")
        norm.to_csv(f"{table_save_path}normality {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # # ANOVA SPHERICITY TEST
        # print('SPHERICITY TEST')
        # print('\n')
        # print('~' * 20)
        # print(f'{iv} x {dv}')
        # spher_all = pg.sphericity(data=df, dv=dv, within=iv, method='mauchly')
        # spher, test_stat, chisq, dof, pval = spher_all
        # print('-' * 20)
        # print(f"{iv} x {dv} Sphericity test:\n{spher} at p-value: {round(pval, 3)}, chi-square: {round(chisq, 3)}, degrees of freedom: {round(dof)}, Test statistic: {round(test_stat)}") # if p-value < 0.05, then the data are not spherically distributed = Multivariate analysis
        # # spher.to_csv(f"{table_save_path}sphericity {df_name} - {iv} x {dv}.csv")
        # print('~' * 20)
        # print('\n')

        # BARTLETTS TESTS
        print("BARTLETT'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        bartlett = pg.homoscedasticity(data=df, dv=dv, group=iv, method='bartlett').round(3) #dv
        equal_var_bartlett = bool(bartlett['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Bartlett's test:\n{bartlett}")
        bartlett.to_csv(f"{table_save_path}bartlett's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')



## Correlation between IVs and Control Variables (Multicollinearity)

### Categorical Language Requirement

In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    # Chi-square
    chisqt = pd.crosstab(df[controls[-2]], df[controls[-1]])
    pearson_r, p_value, dof, expected = scipy.stats.chi2_contingency(chisqt)

    print('+'*120)
    print(f'Dependent Variable: {controls[-2]}\nIndependent Variable: {controls[-1]}')
    print('+'*120)
    print('\n')
    print('~' * 20)
    print(f"The Pearsons's R value: {pearson_r:.3f}\nDegree of freedom: {dof}")
    print('-'*20)
    print(f'Observed Count:\n{chisqt}\n')
    print('-'*20)
    print(f'Expected Count:\n{expected}\n')
    print('-'*20)
    print('-' * 20)
    print(f"Pearsons's R p-value: {p_value:.3f}. Rejected: {p_value < 0.05}")
    print('~' * 20)


### VIF for Dummy IVs and Control Variables

In [ ]:
# compute the vif for all given features
def compute_vif(df, considered_features):

    X = df[considered_features]
    # the calculation of variance inflation requires a constant
    X.insert(0, 'intercept', 1)

    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif.loc[vif['Variable']!='intercept']

    return vif

### VIF for Percentage IVs

In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    for iv_gender_perc, iv_age_perc in tqdm_product(ivs_gender_perc, ivs_age_perc):
        considered_features = [iv_gender_perc, iv_age_perc] + controls[:7]
        considered_features.remove('Platform_Glassdoor')
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


### VIF for Categorical Dummy IVs

In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    considered_features = ivs_dummy[:]
    considered_features.remove('Age_Mixed')
    considered_features.remove('Gender_Mixed')
    vif = compute_vif(df, considered_features)
    print(vif.sort_values('VIF', ascending=False))


In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    for iv_gender_dummy, iv_age_dummy in tqdm_product(ivs_gender_dummy, ivs_age_dummy):
        considered_features = [iv_gender_dummy, iv_age_dummy] + controls[:7]
        considered_features.remove('Platform_Glassdoor')
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    considered_features = ivs_dummy[:] + controls[:7]
    considered_features.remove('Age_Mixed')
    considered_features.remove('Gender_Mixed')
    considered_features.remove('Platform_Glassdoor')
    vif = compute_vif(df, considered_features)
    print(vif.sort_values('VIF', ascending=False))
    vif.to_csv(f'{table_save_path}vif {df_name} - {ivs_dummy} x {controls[:7]}.csv')


In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    if df_name == 'df_manual':
        considered_features_list = [dvs]
    elif df_name == 'df_jobs':
        considered_features_list = [dvs, dvs_prob]

    for considered_features in considered_features_list:
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


# Chi-square

In [ ]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    for iv, dv in tqdm_product(ivs_dummy, dvs):

        # Chi-square
        chisqt = pd.crosstab(df[iv], df[dv])
        pearson_r, p_value, dof, expected = scipy.stats.chi2_contingency(chisqt)
        reject_H0 = p_value > alpha
        if not reject_H0:
            print('+'*120)
            print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
            print('+'*120)
            print('\n')
            print('~' * 20)
            print(f"The Pearsons's R value: {pearson_r:.3f}\nDegree of freedom: {dof}")
            print('-'*20)
            print(f'Observed Count:\n{chisqt}\n')
            print('-'*20)
            print(f'Expected Count:\n{expected}\n')
            print('-'*20)
            print('-' * 20)
            print(f"Pearsons's R p-value: {p_value:.3f}. Rejected: {p_value < 0.05}")
            print('~' * 20)

            # # Plot acceptance region distribution
            # x = np.linspace(0, 10, 100)
            # fig,ax = plt.subplots(1,1, figsize=(15,10))
            # #plotting vertical line for critical value 
            # plt.axvline(x=scipy.stats.chi2.isf(0.05,dof), ymin=0, ymax= 0.3,label='X-Critical',color='black')
            # #plotting vertical line for calculated value. 
            # plt.axvline(x=stat, ymin=0, ymax= 0.3,label='X-calculated',color='blue')
            # #plotting distribution graph for our calculated degrees of freedom
            # ax.plot(x, scipy.stats.chi2.pdf(x, dof), label=f'df = {str(dof)}', color='red')
            # ax.set_xlabel('Value',fontsize=12, fontweight='bold')
            # ax.set_ylabel('Probability Distribution',fontsize=12,fontweight='bold')
            # ax.set_title(f'Chi-Square Distribution for {dv} x {iv}',fontsize=16,fontweight='bold')
            # plt.xlim(0, 10)
            # plt.ylim(0, 0.6)
            # plt.ion()
            # plt.legend()
            # plt.show()


# ANOVA

In [ ]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_manual':
        dvs_ = dvs
    elif df_name == 'df_jobs':
        dvs_ = dvs_all

    for iv, dv in tqdm_product(ivs, dvs_):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # LEVENE'S TESTS
        print("LEVENE'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        levene = pg.homoscedasticity(data=df, dv=dv, group=iv, method='levene').round(3) #dv
        equal_var_levene = bool(levene['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Levene's test:\n{levene}")
        levene.to_csv(f"{table_save_path}levene's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # SCIPY ANOVAS
        print('ANOVA SIGNIFICANCE')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        f_statistic, p_value = f_oneway(
            df[dv][df[iv] == ivs_dict[iv][0]],
            df[dv][df[iv] == ivs_dict[iv][1]],
            df[dv][df[iv] == ivs_dict[iv][2]]
        )
        reject_H0 = p_value > alpha
        print('-' * 20)
        print(f'One-way ANOVA p-value: {p_value}. Rejected: {p_value < 0.05}')
        print('~' * 20)

        if not reject_H0:
            # INTERACTION MODEL
            print(f'INTEACTION ANOVA {dv}')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            formula = f'{dv} ~ C({ivs[0]})*C({ivs[1]})'
            model = ols(data = df, formula = formula).fit()
            anova_interaction = sm.stats.anova_lm(model, typ=2).round(3)
            print(f'{iv} x {dv} ANOVA INTERACTION:\n{anova_interaction}')
            print('~' * 20)
            print('\n')

            if equal_var_levene:
                # ONE-WAY ANOVA
                print('ONE-WAY ANOVA')
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova1 = pg.anova(data=df, dv=dv, between=iv, detailed=True).round(3)
                print(f'{iv} x {dv} ONE-WAY ANOVA:\n{anova1}')
                anova1.to_csv(f'{table_save_path}one-way anova {df_name} - {iv} x {dv}.csv')
                print('~' * 20)
                print('\n')

                # TWO-WAY ANOVA
                print('TWO-WAY ANOVA')
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova2 = pg.anova(data=df, dv=dv, between=ivs, detailed=True).round(3)
                print(f'{iv} x {dv} TWO-WAY ANOVA:\n{anova2}')
                anova2.to_csv(f'{table_save_path}two-way anova {df_name} - {ivs[0]} and {ivs[1]} x {dv}.csv')
                print('~' * 20)
                print('\n')

                # TUKEY POST HOC
                print("POST HOC TUKEY'S ANOVA")
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova_pairwise_tukey = pg.pairwise_tukey(
                    data=df, dv=dv, between=iv, effsize='eta-square'
                ).round(3)
                pg.print_table(anova_pairwise_tukey)
                anova_pairwise_tukey.to_csv(f'{table_save_path}post hoc tukey {df_name} - {iv} x {dv}.csv')
                print('~' * 20)
                print('\n')

            if not equal_var_levene:
                # WELCH ANOVA
                print('WELCH ANOVA')
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova_welch = pg.welch_anova(data=df, dv=dv, between=iv).round(3)
                pg.print_table(anova_welch)
                anova_welch.to_csv(f'{table_save_path}welch anova {df_name} - {iv} x {dv}.csv')
                print('~' * 20)
                print('\n')

                # ## INTERACTION ANOVA
                # print('WELCH INTERACTION ANOVA')
                # print('\n')
                # print('~' * 20)
                # print(f'{ivs[0]} and {ivs[1]} x {dv}')
                # anova_welch_interaction = pg.welch_anova(data=df, dv=dv, between=ivs).round(3)
                # pg.print_table(anova_welch_interaction)
                # anova_welch_interaction.to_csv(f'{table_save_path}welch interaction anova {df_name} - {ivs} x {dv}.csv')
                # print('~' * 20)
                # print('\n')

                # KRUSKAL-WALLIS ANOVA
                print('KRUSKAL-WALLIS ANOVA')
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova_kruskal = pg.kruskal(data=df, dv=dv, between=iv).round(3)
                pg.print_table(anova_kruskal)
                anova_kruskal.to_csv(f'{table_save_path}kruskal-wallis anova {df_name} - {iv} x {dv}.csv')
                print('~' * 20)
                print('\n')

                # ## INTERACTION ANOVA
                # print('KRUSKAL-WALLIS INTERACTION ANOVA')
                # print('\n')
                # print('~' * 20)
                # print(f'{ivs[0]} and {ivs[1]} x {dv}')
                # anova_kruskal_interaction = pg.kruskal(data=df, dv=dv, between=ivs).round(3)
                # pg.print_table(anova_kruskal_interaction)
                # anova_kruskal_interaction.to_csv(f'{table_save_path}kruskal-wallis interaction anova {df_name} - {ivs} x {dv}.csv')
                # print('~' * 20)
                # print('\n')

                # GAMES HOWELL POST HOC
                print('POST HOC GAMES HOWELL ANOVA')
                print('\n')
                print('~' * 20)
                print(f'{iv} x {dv}')
                anova_games_posthoc = pg.pairwise_gameshowell(
                    data=df, dv=dv, between=iv, effsize='eta-square'
                ).round(3)
                pg.print_table(anova_games_posthoc)
                anova_games_posthoc.to_csv(f'{table_save_path}post hoc gameshowell {df_name} - {iv} x {dv}.csv')
                print('~' * 20)
                print('\n')
                print('+'*120)
                print('\n')


# Regressions

## Logistic Regression

In [ ]:
# Edit variable names for formula use: remove % and replace spaces with underscores
ivs_perc_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc))
print(ivs_perc_)
controls_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), controls))
print(controls_)

In [ ]:
dataframes_ = {
    'df_manual_': df_manual.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_manual.columns}),
    'df_jobs': df_jobs.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_jobs.columns}),
}


### Logistic Regression with Social Category Dummies

In [ ]:
%%time
# Logistic Regression for 0:1 Warmth and Competence x 0:1 Gender and Age
for df_name, df in dataframes_.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\n\nINDEPENDENT VARIABLE: {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]}')
        print('\n')
        print('+'*120)

        # model = sm.Logit(endog=df[dv], exog=df[ivs_perc], data=df)
        # formula = f'{dv} ~ {ivs_dummy[0]}*{ivs_dummy[3]} + {ivs_dummy[0]}*{ivs_dummy[4]} + {ivs_dummy[0]}*{ivs_dummy[5]} + {ivs_dummy[1]}*{ivs_dummy[3]} + {ivs_dummy[1]}*{ivs_dummy[4]} + {ivs_dummy[1]}*{ivs_dummy[5]} + {ivs_dummy[2]}*{ivs_dummy[3]} + {ivs_dummy[2]}*{ivs_dummy[4]} + {ivs_dummy[2]}*{ivs_dummy[5]} + {controls_[0]} + {controls_[1]} + C({controls_[2]}) + C({controls_[3]})'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[1]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[4]} + {ivs_dummy[5]} + {controls_[0]} + {controls_[1]} + {controls_[2]} + {controls_[3]} + {controls_[4]} + {controls_[5]}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]} + {controls_[0]} + {controls_[1]} + {controls_[2]} + {controls_[3]} + {controls_[4]} + {controls_[5]}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[1]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[4]} + {ivs_dummy[5]}'

        formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]}'

        print('-'*20)
        print(f'Using formula: {formula}')
        print('-'*20)

        with contextlib.suppress(np.linalg.LinAlgError):
            model = smf.logit(formula=formula, data=df)
            results = model.fit()
            df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))

            # Display Results
            print('~'*20)
            print('\n')
            print(f'SUMMARY RESULTS:\n{results.summary()}\n')
            print('~'*20)
            # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
            # print('-'*20)
            # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
            # print('-'*20)
            # print(f'COEFFICIENT:\n{results.params}')
            # print('-'*20)
            # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
            # print(f'P-VALUES:\n{results.pvalues}')
            # print('-'*20)
            # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
            # print(f'AIC:\n{results.aic:.2f}')
            # print('-'*20)
            # print(f'BIC:\n{results.bic:.2f}')
            # print('-'*20)
            # print(f'Coehn\'s F2:\n{results.prsquared:.3f}')
            # print('-'*20)

            # save results
            df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
            df_summary_results.to_csv(f'{table_save_path}logistic regression on categories {df_name} - {dv} x Social Category Percentages.csv', index=False)


### Logistic Regression with Social Category percentage per Sector

In [ ]:
%%time
# Logistic Regression for 0:1 Warmth and Competence x percentage Gender and Age
for df_name, df in dataframes_.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\n\nINDEPENDENT VARIABLE: {ivs_perc_}')
        print('\n')
        print('+'*120)

        # model = sm.Logit(endog=df[dv], exog=df[ivs_perc], data=df)
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + {controls_[2]} + {controls_[3]} + {controls_[4]} + {controls_[5]}'

        formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + {controls_[2]} + {controls_[3]}'

        print('-'*20)
        print(f'Using formula: {formula}')
        print('-'*20)

        model = smf.logit(formula=formula, data=df)
        results = model.fit()

        # Display Results
        print('~'*20)
        print('\n')
        print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        print('~'*20)
        # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # print('-'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # print('-'*20)
        # print(f'COEFFICIENT:\n{results.params}')
        # print('-'*20)
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # print(f'P-VALUES:\n{results.pvalues}')
        # print('-'*20)
        # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # print(f'Coehn\'s F2:\n{results.prsquared:.3f}')
        # print('-'*20)

        # save results
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
        df_summary_results.to_csv(f'{table_save_path}logistic regression on percentages {df_name} - {dv} x Social Category Percentages.csv', index=False)


## Linear Regression

In [ ]:
# Edit variable names for formula use: remove % and replace spaces with underscores
ivs_perc_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc))
print(ivs_perc_)
controls_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), controls))
print(controls_)

In [ ]:
dataframes_ = {
    'df_manual_': df_manual.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_manual.columns}),
    'df_jobs': df_jobs.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_jobs.columns}),
}


### OLS Regression

In [ ]:
for df_name, df in dataframes_.items():

    if df_name == 'df_manual_':
        dvs_ = dvs
    elif df_name == 'df_jobs_':
        dvs_ = dvs_all

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs_):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\n\nINDEPENDENT VARIABLE: {ivs_perc_}')
        print('\n')
        print('+'*120)

        # constant = sm.add_constant(df[ivs_perc_])
        # model = sm.OLS(df[dv], constant)
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + C({controls_[2]}) + C({controls_[3]})'
        formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + {controls_[2]} + {controls_[3]}'

        print('-'*20)
        print(f'Using formula: {formula}')
        print('-'*20)

        model = smf.ols(formula=formula, data=df)
        results = model.fit()

        # Display Results
        print('~'*20)
        print('\n')
        print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        print('~'*20)
        # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # print('-'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # print('-'*20)
        # print(f'COEFFICIENT:\n{results.params}')
        # print('-'*20)
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # print(f'P-VALUES:\n{results.pvalues}')
        # print('-'*20)
        # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # print(f'Coehn\'s F2:\n{results.rsquared_adj:.3f}')
        print('-'*20)
        table = sm.stats.anova_lm(results, typ=2)
        print(f'ANOVA:\n{table}')
        print('-'*20)

        # # Boxplot
        # boxplot = df.boxplot([dv], by = [ivs_perc_[2], ivs_perc_[0]],
        #                     figsize = (16, 9),
        #                     showmeans = True,
        #                     notch = True)

        # boxplot.set_xlabel('Categories')
        # boxplot.set_ylabel(dv)
        # # Creating a path to save the plot.
        # plt.ion()
        # plt.show()
        # plt.pause(.001)
        # # for image_save_format in ['eps', 'png', 'svg']:
        # #     save_path = f'{plot_save_path}Probability Boxplot - {df_name} - {dv} x Social Category Percentages.{image_save_format}'
        # #     boxplot.figure.savefig(
        # #         save_path, format=image_save_format,
        # #     )
        # plt.close()


### Multi-level OLS Regression

In [ ]:
# Edit variable names for formula use: remove % and replace spaces with underscores
ivs_perc_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc))
print(ivs_perc_)
controls_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), controls))
print(controls_)

In [ ]:
dataframes_ = {
    'df_manual_': df_manual.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_manual.columns}),
    'df_jobs': df_jobs.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_jobs.columns}),
}


In [ ]:
for df_name, df in dataframes_.items():

    if df_name == 'df_jobs':
        dvs_ = dvs_all

    df['Intercept'] = 1

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\n\nINDEPENDENT VARIABLE: {ivs_perc_}')
        print('\n')
        print('+'*120)

        save_name = f'Multilevel Logistic Regression {df_name} - {list(iter(ivs_dict))[0]} + {list(iter(ivs_dict))[1]} x {dv}'
        # endog = df[dv]
        # exog0 = df[['Intercept', f'{list(iter(ivs_dict))[0]}']]
        # exog1 = df[['Intercept', f'{list(iter(ivs_dict))[1]}']]
        # iv_1 = list(iter(ivs_dict))[0]
        # iv_1_treatment = ivs_dict[iv_1][0]
        # iv_2 = list(iter(ivs_dict))[1]
        # iv_2_treatment = ivs_dict[iv_2][0]

        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + C({controls_[2]}) + C({controls_[3]})'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + C({controls_[2]}) + C({controls_[3]}) + C({dvs[0]}):C({dvs[1]})'
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {controls_[0]} + {controls_[1]} + C({controls_[2]}) + C({controls_[3]})'

        formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_[1]} + {controls_[2]} + {controls_[3]}'

        print('-'*20)
        print(f'Using formula: {formula}')
        print('-'*20)

        vc_formula = {f'{controls_[1]}': f'0 + {controls_[1]}'}
        re_formula = f'1 + {controls_[1]}'

        model = smf.mixedlm(formula=formula, data=df, groups='Job_ID',) #vc_formula=vc_formula, re_formula=re_formula)
        results = model.fit(method='lbfgs')
        gradient = model.score(results.params_object)

        # Display Results
        print('~'*20)
        print(f'Gradient:\n{gradient}')
        print('\n')
        print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        print('~'*20)
        # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # print('-'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # print('-'*20)
        # print(f'COEFFICIENT:\n{results.params}')
        # print('-'*20)
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # print(f'P-VALUES:\n{results.pvalues}')
        # print('-'*20)
        # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # print(f'Coehn\'s F2:\n{results.rsquared_adj:.3f}')
        # print('-'*20)
        # table = sm.stats.anova_lm(results, typ=2)
        # print(f'ANOVA:\n{table}')
        # print('-'*20)

        # df_results = pd.DataFrame(index=['Descriptives', 'Results'], columns=[f'{save_name}'])
        # df_results[f'{save_name}']['Descriptives'] = results.summary().tables[0]
        # df_results[f'{save_name}']['Results'] = results.summary().tables[1]

        # df_results.to_csv(f'{table_save_path}{save_name.split(" results")[0].lower() + " results" + save_name.split(" results")[1]}.csv', header=True, index=True, index_label=['Index col: Descriptives and Results'])

        # # Normality Tests (https://www.pythonfordatascience.org/mixed-effects-regression-python/)
        # ## Residual and Kernal Density Estimate (KDE) Plot for Homoskedasticity
        # fig = plt.figure(figsize = (16, 9))

        # ax = sns.distplot(results.resid, hist = True, kde_kws = {"shade" : True, "lw": 1}, fit = scipy.stats.norm, kde=True, palette='colorblind')

        # ax.set_title(f"Kernal Density Estimate (KDE) Plot of Model Residuals (Blue) and Normal Distribution (Black)\n{save_name}")
        # ax.set_xlabel("Residuals")
        # plt.ion()
        # fig.show('notebook')
        # plt.pause(.001)

        # # Q-Q Plot
        # fig = plt.figure(figsize = (16, 9))
        # ax = fig.add_subplot(111)

        # qq = sm.qqplot(results.resid, dist = scipy.stats.norm, line = 's', ax = ax, color='blue', markerfacecolor='blue')
        # ax.set_title(f"Q-Q Plot\n{save_name}",fontsize=15)
        # ax.xaxis.get_label().set_fontsize(12)
        # ax.yaxis.get_label().set_fontsize(12)
        # ax.get_lines()[0].set_color('black')
        # ax.get_lines()[0].set_linewidth('2')
        # ax.get_lines()[1].set_color('black')
        # ax.get_lines()[1].set_linewidth('2')
        # plt.ion()
        # fig.show('notebook')
        # plt.pause(.001)

        # # Test of Normality
        # norm = scipy.stats.normaltest(results.resid)

        # print('='*80)
        # print(f'{dv} Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Skewness-Kurtosis Test of Normality
        # norm_sk = scipy.stats.kurtosistest(results.resid)

        # print('='*80)
        # print(f'{dv} Skewness-Kurtosis Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Shapir-Wilk Test of Normality
        # norm_res = scipy.stats.shapiro(results.resid)

        # print('='*80)
        # print(f'{dv} Shapir-Wilk Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_res)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Anderson-Darling Test of Normality
        # norm_and = scipy.stats.anderson(results.resid)

        # print('='*80)
        # print(f'{dv} Anderson-Darling Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_and)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Residuals versus Fitted values (RVF) Plot for Homoskedasticity
        # fig = plt.figure(figsize = (16, 9))

        # ax = sns.scatterplot(y = results.resid, x = results.fittedvalues, palette='colorblind')

        # ax.set_title(f"Residuals versus Fitted values (RVF) Plot\n{save_name}")
        # ax.set_xlabel("Fitted Values")
        # ax.set_ylabel("Residuals")
        # plt.ion()
        # fig.show('notebook')
        # plt.pause(.001)

        # # White’s Lagrange Multiplier Test for Heteroscedasticity
        # het_white_res = het_white(results.resid, results.model.exog)

        # het_white_labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]

        # print('='*80)
        # print('White’s Lagrange Multiplier Test for Heteroscedasticity')
        # print('-'*80)
        # for key, val in dict(zip(het_white_labels, het_white_res)).items():
        #     print(key, val)
        # print('\n')
        # print('\n')
        # print('+'*120)
        # print('\n')


# Specification Curve Analysis

In [ ]:
# Make dicts of models and IVs
sm_models = {
    'logistic': sm.Logit,
    'OLS': sm.OLS,
}

ivs_for_spec = {
    'dummy': ivs_dummy,
    'percentages': ivs_perc,
    'all': ivs_dummy_and_perc,
}


In [ ]:
%%time
for (df_name, df), (model_name, model), (iv_type, ivs) in tqdm_product(dataframes.items(), sm_models.items(), ivs_for_spec.items()):

    if df_name == 'df_manual':
        dvs_ = dvs
    elif df_name == 'df_jobs':
        dvs_ = dvs_all

    print(f'{"="*5} {model_name.upper()} REGRESSION SPECIFICATION MODE RESULTS FOR {df_name} USING {iv_type.upper()} {"="*5}')
    print(f'Running specification curve analysis with:\nDEPENDENT VARIABLES = {dvs_}\nINDEPENDENT VARIABLES = {ivs}\nCONTROLS = {controls}')

    with contextlib.suppress(np.linalg.LinAlgError):
        sc = specy.SpecificationCurve(df=df, y_endog=dvs_, x_exog=ivs, controls=controls[:6])
        sc.fit(estimator=model)
        df_results = sc.df_r

        # Plot and save
        for iv, dv in tqdm_product(ivs, dvs_):
            print('~'*80)
            print(f'\n{"="*5} RESULTS FOR {iv.title()} ON {dv.title()} {"="*5}\n')
            print('~'*80)

            for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
                if iv_type == 'dummy':
                    plot_title = f"{dv.title()} x {iv.split('_')[1].title()}-dominated Sectors"
                elif iv_type == 'percentages':
                    plot_title = f"{dv.title()} x {' '.join(iv.split('_')[-2:])}"
                save_path = f'{plot_save_path}{df_name} - Specification Curve - {iv} x {dv}.{image_save_format}'
                # Use following if not using forked specification_curve
                # sc.plot(preferred_spec=[iv, dv], save_path=save_path,)
                sc_fig = sc.plot(
                    preferred_spec=[iv, dv].extend(controls),
                    save_path=save_path,
                    show_plot=False,
                    return_fig=True,
                    plot_title=plot_title
                )

        # Get statsmodels results and save
        ## Get controls mask
        controls_mask = df_results['Specification'].apply(lambda x: all(control in x for control in controls))
        ## Get gender only results
        gender_mask = df_results['Specification'].apply(lambda x: any(item for item in ivs[:-len(ivs)//2] if item in x and len(x) == 2))
        df_results_gender = df_results[gender_mask]
        if df_results_gender[controls_mask].empty:
            print('No specification with Gender and all controls.')
        else:
            df_results_gender = df_results_gender[controls_mask]
        # Get age only results
        age_mask = df_results['Specification'].apply(lambda x: any(item for item in ivs[len(ivs)//2:] if item in x and len(x) == 2))
        df_results_age = df_results[age_mask]
        if df_results_age[controls_mask].empty:
            print('No specification with Age and all controls.')
        else:
            df_results_age = df_results_age[controls_mask]

        for df in [df_results_gender, df_results_age]:
            for idx, row in df.iterrows():
                for dv_iv in row['Specification']:
                    if dv_iv in ivs_dummy_and_perc:
                        iv_name = dv_iv
                    elif dv_iv in dvs_:
                        dv_name = dv_iv
                print('\n')
                print('+'*20)
                print(f'{dv_name} x {iv_name}\n')
                print('+'*20)
                print(f'{row["Results"].summary()}')
                print('-'*20)

                # Save results to file
                df_to_save = pd.DataFrame(csv.reader(row['Results'].summary().as_csv().split('\n'), delimiter=','))
                df_to_save.to_csv(f'{table_save_path}{model_name} specification curve {df_name} - {iv_type} - {dv_name} x {iv_name}.csv', index=False)

        # Top 10 significant highest coefficients
        df_coeff_p = df_results.loc[sc.df_r['coeff_pvals'] < 0.05].sort_values(by=['Coefficient'], ascending=False)
        print(f"Top 10 significant coefficients:\n{df_coeff_p[['x_exog', 'y_endog', 'coeff_pvals', 'Coefficient', 'conf_int', 'pvalues']].head(10)}")

        print(f'{"="*5} END OF RESULTS FOR {iv.title()} {"="*5}')
        print('~'*80, '\n')


In [ ]:
# %%time
# # Logistic Specification Curve Analysis for 0:1 Warmth and Competence x percentage Gender and Age
# for df_name, df in dataframes.items():

#     print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')
#     print(f'Running specification curve analysis with:\nDEPENDENT VARIABLES = {dvs}\nINDEPENDENT VARIABLES = {ivs_dummy}\nCONTROLS = {controls}')

#     sc = specy.SpecificationCurve(df=df, y_endog=dvs, x_exog=ivs_dummy, controls=controls)
#     sc.fit(estimator=sm.Logit)
#     df_results = sc.df_r

#     # Plot and save
#     for iv_dummy, dv in tqdm_product(ivs_dummy, dvs):
#         print(f'{"="*5} RESULTS FOR {iv_dummy.title()} ON {dv.title()} {"="*5}')

#         for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
#             save_path = f'{plot_save_path}Specification Curve - {iv_dummy} x {dv}.{image_save_format}'
#             # Use following if not using forked specification_curve
#             # sc.plot(preferred_spec=[iv, dv], save_path=save_path,)
#             sc_fig  = sc.plot(
#                 preferred_spec=[iv_dummy, dv],
#                 save_path=save_path,
#                 show_plot=False,
#                 return_fig=True,
#                 plot_title=f"{dv.title()} x {iv_dummy.split('_')[1].title()}-dominated Sectors"
#             )
#         print(sc_fig)

#     # Get statsmodels results and save
#     ## Get gender only results
#     gender_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Gender_Female_% per Sector', 'Gender_Male_% per Sector'] if item in x and len(x) == 2))
#     df_results_gender = df_results[gender_mask]
#     # Get age only results
#     age_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Age_Older_% per Sector', 'Age_Younger_% per Sector'] if item in x and len(x) == 2))
#     df_results_age = df_results[age_mask]

#     for df in [df_results_gender, df_results_age]:
#         for idx, row in df.iterrows():
#             for dv_iv in row["Specification"]:
#                 if dv_iv in ivs_dummy:
#                     iv_name = dv_iv
#                 elif dv_iv in dvs:
#                     dv_name = dv_iv
#             print('\n')
#             print('+'*20)
#             print(f'{dv_name} x {iv_name}\n')
#             print('+'*20)
#             print(f'{row["Results"].summary()}')
#             print('-'*20)

#             # Save results to file
#             df_to_save = pd.DataFrame(csv.reader(row["Results"].summary().as_csv().split('\n'), delimiter=','))
#             df_to_save.to_csv(f'{table_save_path}logistic specification curve dummy - {dv_name} x {iv_name} {df_name}.csv', index=False)

#     # Top 10 significant highest coefficients
#     df_coeff_p = df_results.loc[sc.df_r['coeff_pvals'] < 0.05].sort_values(by=['Coefficient'], ascending=False)
#     print(f"Top 10 significant coefficients:\n{df_coeff_p[['x_exog', 'y_endog', 'coeff_pvals', 'Coefficient', 'conf_int', 'pvalues']].head(10)}")

#     print(f'{"="*5} END OF RESULTS FOR {iv_dummy.title()} {"="*5}')


In [ ]:
# %%time
# # Logistic Specification Curve Analysis for 0:1 Warmth and Competence x percentage Gender and Age
# for df_name, df in dataframes.items():

#     print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')
#     print(f'Running specification curve analysis with:\nDEPENDENT VARIABLES = {dvs}\nINDEPENDENT VARIABLES = {ivs_perc}\nCONTROLS = {controls}')

#     sc = specy.SpecificationCurve(df=df, y_endog=dvs, x_exog=ivs_perc, controls=controls)
#     sc.fit(estimator=sm.Logit)
#     df_results = sc.df_r

#     # Plot and save
#     for iv_perc, dv in tqdm_product(ivs_perc, dvs):
#         print(f'{"="*5} RESULTS FOR {iv_perc.title()} ON {dv.title()} {"="*5}')

#         for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
#             save_path = f'{plot_save_path}Specification Curve - {iv_perc} x {dv}.{image_save_format}'
#             # Use following if not using forked specification_curve
#             # sc.plot(preferred_spec=[iv, dv], save_path=save_path,)
#             sc_fig  = sc.plot(
#                 preferred_spec=[iv_perc, dv],
#                 save_path=save_path,
#                 show_plot=False,
#                 return_fig=True,
#                 plot_title=f"{dv.title()} x {' '.join(ivs_perc[0].split('_')[-2:])}"
#             )
#         print(sc_fig)

#     # Get statsmodels results and save
#     ## Get gender only results
#     gender_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Gender_Female_% per Sector', 'Gender_Male_% per Sector'] if item in x and len(x) == 2))
#     df_results_gender = df_results[gender_mask]
#     # Get age only results
#     age_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Age_Older_% per Sector', 'Age_Younger_% per Sector'] if item in x and len(x) == 2))
#     df_results_age = df_results[age_mask]

#     for df in [df_results_gender, df_results_age]:
#         for idx, row in df.iterrows():
#             for dv_iv in row["Specification"]:
#                 if dv_iv in ivs_perc:
#                     iv_name = dv_iv
#                 elif dv_iv in dvs:
#                     dv_name = dv_iv
#             print('\n')
#             print('+'*20)
#             print(f'{dv_name} x {iv_name}\n')
#             print('+'*20)
#             print(f'{row["Results"].summary()}')
#             print('-'*20)

#             # Save results to file
#             df_to_save = pd.DataFrame(csv.reader(row["Results"].summary().as_csv().split('\n'), delimiter=','))
#             df_to_save.to_csv(f'{table_save_path}logistic specification curve percentages - {dv_name} x {iv_name} {df_name}.csv', index=False)

#     # Top 10 significant highest coefficients
#     df_coeff_p = df_results.loc[sc.df_r['coeff_pvals'] < 0.05].sort_values(by=['Coefficient'], ascending=False)
#     print(f"Top 10 significant coefficients:\n{df_coeff_p[['x_exog', 'y_endog', 'coeff_pvals', 'Coefficient', 'conf_int', 'pvalues']].head(10)}")

#     print(f'{"="*5} END OF RESULTS FOR {iv_perc.title()} {"="*5}')
